In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

In [2]:
df_arab = pd.read_csv('../testing/arab_links_script_2.csv')
df_jordan = pd.read_csv('../testing/jordan_links_script_2.csv')
df_etihad = pd.read_csv('../testing/etihad_links_script_2.csv')
df_housing = pd.read_csv('../testing/housing_links_script_2.csv')
df_central = pd.read_csv('../testing/central_links_script.csv')
df_faq = pd.read_csv('../testing/faq.csv')
df_full_housing = pd.read_csv('../testing/new_script.csv')

In [3]:
df_full_housing = df_full_housing.head(600)

In [4]:
df_full_housing

,Name,URL
0,The Housing Bank for Trade and Finance - Housi...,https://hbtf.com/en
1,Contact Us - Housing Bank,https://hbtf.com/en/contact
2,Retail - Housing Bank,https://hbtf.com/en/retail
3,Edge - Housing Bank,https://hbtf.com/en/retail/programs/edge
4,Corporate Banking Sector - Housing Bank,https://hbtf.com/en/corporate
...,...,...
595,News - Housing Bank,https://hbtf.com/en/news?page=17
596,News - Housing Bank,https://hbtf.com/en/news?page=18
597,News - Housing Bank,https://hbtf.com/en/news?page=19
598,News - Housing Bank,https://hbtf.com/en/news?page=20


In [5]:
# dfs = [df_arab,df_jordan,df_etihad]
# dfs = {'arab':df_arab,
#        'jordan':df_jordan,
#        'etihad':df_etihad,
#        'housing':df_housing
#       }
dfs = {'full_housing':df_full_housing}

In [6]:
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
#     "Accept-Language": "en-US,en;q=0.9",
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#     "Connection": "keep-alive"
# }

# session = requests.Session()
# session.headers.update(headers)

In [7]:
OUTPUT_CSV = 'scraped_bank_data.csv'
SCROLL = False
WAIT_TIME = 0.1

In [8]:
options = Options()
options.add_argument('--headless=new')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

In [9]:
def scroll_page():
    last_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(WAIT_TIME/2)
        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == last_height:
            break
        last_height= new_height

In [10]:
# for bank, df in dfs.items():
#     names, urls = df['Name'], df['URL']
#     size = len(names)
#     df['Text'] = None
#     for i in range(size):
#         url = urls[i]
#         name = names[i]
#         print(f'[ INFO: {bank} bank {i + 1}/{size}] Visiting {name} at {url}')

#         try:
#             driver.get(url)
#             time.sleep(WAIT_TIME)
#             if SCROLL:
#                 scroll_page()
#                 time.sleep(WAIT_TIME)
#             html = driver.page_source

#             soup = BeautifulSoup(html, 'html.parser')
#             for tag in soup(['header','footer','nav','aside','script','style']):
#                 tag.decompose()
#             # text = soup.get_text(separator='\n')
#             for tag in soup.select('.social, .contact, #footer, #header'):
#                 tag.decompose()

#             text = soup.get_text(separator='\n')
#             text = text.replace('\xa0',' ')
#             text = '\n'.join({line.strip() for line in text.splitlines() if line.strip()})
#             if len(text) >= 1000:
#                 df.at[i,'Text'] = text  

#             print(f'[INFO: {bank} bank {i + 1}/{size}] Scraped {len(text)} from {url}')
#         except Exception as e:
#             print(f'[INFO: {bank} bank {i + 1}/{size}] Error scraping {url}: {e}')

#     df = df[df['Text'] != None]
#     output_file = f'{bank}_scraped_data_test.csv'
#     df.to_csv(output_file)
#     print(f'[INFO: {bank} bank {i + 1}/{size}] Saved Data in {output_file}')
                

In [11]:
loop_time = time.time()

for bank, df in dfs.items():
    bank_time = time.time()
    names, urls = df['Name'], df['URL']
    size = len(names)
    df['Text'] = None
    for i in range(size):
        url = urls[i]
        name = names[i]
        print(f'[ INFO: {bank} bank {i + 1}/{size}] Visiting {name} at {url}')

        try:
            r = requests.get(url,timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.text,'html.parser')

            for tag in soup(['header','footer','nav','aside','script','style']):
                tag.decompose()

            text = soup.get_text(separator='\n')
            text = '\n'.join([line.strip() for line in text.splitlines() if line.strip()])

            if len(text) >= 1000:
                df.at[i,'Text'] = text

            print(f'[INFO: {bank} bank {i + 1}/{size}] Scraped {len(text)} from {url}')

        except Exception as e:
            print(f'[INFO: {bank} bank {i + 1}/{size}] Error scraping {url}: {e}')

    df = df[df['Text'] != None]
    output_file = f'{bank}_scraped_data.csv'
    df.to_csv(output_file)
    bank_end = time.time() - bank_time
    print(f'[INFO: {bank} time {bank_end:.2f}s] Saved Data in {output_file}')

loop_end = time.time() - loop_time
print(f'[INFO] Ended Scraping. Time taken {loop_end:.2f}s')
                

[ INFO: full_housing bank 1/600] Visiting The Housing Bank for Trade and Finance - Housing Bank at https://hbtf.com/en
[INFO: full_housing bank 1/600] Scraped 10845 from https://hbtf.com/en
[ INFO: full_housing bank 2/600] Visiting Contact Us - Housing Bank at https://hbtf.com/en/contact
[INFO: full_housing bank 2/600] Scraped 9204 from https://hbtf.com/en/contact
[ INFO: full_housing bank 3/600] Visiting Retail - Housing Bank at https://hbtf.com/en/retail
[INFO: full_housing bank 3/600] Scraped 10636 from https://hbtf.com/en/retail
[ INFO: full_housing bank 4/600] Visiting Edge - Housing Bank at https://hbtf.com/en/retail/programs/edge
[INFO: full_housing bank 4/600] Scraped 11783 from https://hbtf.com/en/retail/programs/edge
[ INFO: full_housing bank 5/600] Visiting Corporate Banking Sector - Housing Bank at https://hbtf.com/en/corporate
[INFO: full_housing bank 5/600] Scraped 10484 from https://hbtf.com/en/corporate
[ INFO: full_housing bank 6/600] Visiting Treasury & Investment - H

In [12]:
driver.quit()